In [5]:
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
import pandas as pd

In [52]:
import numpy as np

In [4]:
!pip install sparse_dot_topn

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for sparse-dot-topn: filename=sparse_dot_topn-0.3.1-cp37-cp37m-linux_x86_64.whl size=1582098 sha256=50dbd37e444df1ddb12769e9b58336c3329caece713cd2f918ab53f0725f85c0
  Stored in directory: /root/.cache/pip/wheels/3b/3e/02/4ee8cb28ed8b608d530bc43402518a895db8ce89aff8ca4e1f
Successfully built sparse-dot-topn


In [44]:
df = pd.read_csv('/content/drive/MyDrive/Seattle_Hotels_Duplicates.csv', encoding="latin-1")

In [45]:
df.head(3)

,name,address
0,Hilton Garden Inn Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA"
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA"
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101"


In [46]:
df.name.value_counts()

Hilton Garden Inn Seattle Downtown                     2
Ace Hotel Seattle                                      2
citizenM Seattle South Lake Union hotel                2
Hyatt Regency Lake Washington At SeattleS Southport    2
Quality Inn & Suites Seattle Center                    2
                                                      ..
DoubleTree by Hilton Hotel Seattle Airport             1
Radisson Hotel Seattle Airport                         1
Crowne Plaza Seattle Airport                           1
Hilton Seattle Airport & Conference Center             1
MarQueen Hotel                                         1
Name: name, Length: 155, dtype: int64

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
df.loc[df['name'] == 'Roy Street Commons']

,name,address
82,Roy Street Commons,"621 12th Ave E, Seattle, WA 98102"
90,Roy Street Commons,"621 12th Avenue East, Seattle, Washington 98102"


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer 
df['name_address'] = df['name'] + ' ' + df['address']
name_address = df['name_address']
vectorizer = TfidfVectorizer(analyzer = 'char', ngram_range=(1, 4), sublinear_tf=True)
tf_idf_matrix = vectorizer.fit_transform(name_address)

In [50]:
tf_idf_matrix

<168x4957 sparse matrix of type '<class 'numpy.float64'>'
	with 32557 stored elements in Compressed Sparse Row format>

In [53]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
  
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 5)

In [54]:
matches

<168x168 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [55]:
def get_matches_df(sparse_matrix, name_vector, top=840):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

matches_df = get_matches_df(matches, name_address)

In [56]:
matches_df[matches_df['similarity'] < 0.99999].sort_values(by=['similarity'], ascending=False).head(30)

,left_side,right_side,similarity
831,Pike's Place Lux Suites by Barsala 2rd Ave and...,Pike's Place Lux Suites by Barsala 2nd Ave and...,0.932024
826,Pike's Place Lux Suites by Barsala 2nd Ave and...,Pike's Place Lux Suites by Barsala 2rd Ave and...,0.932024
1,Hilton Garden Inn Seattle Downtown 1821 Boren ...,Hilton Garden Inn Seattle Downtown 1821 Boren ...,0.911902
46,Hilton Garden Inn Seattle Downtown 1821 Boren ...,Hilton Garden Inn Seattle Downtown 1821 Boren ...,0.911902
586,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.889089
551,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.889089
561,"Renaissance Seattle Hotel 515 Madison Street, ...","Renaissance Seattle Hotel 515 Madison St, Seat...",0.874515
546,"Renaissance Seattle Hotel 515 Madison St, Seat...","Renaissance Seattle Hotel 515 Madison Street, ...",0.874515
341,Home2 Suites by Hilton Seattle Airport 380 Upl...,Home2 Suites by Hilton Seattle Airport 380 Upl...,0.873906
346,Home2 Suites by Hilton Seattle Airport 380 Upl...,Home2 Suites by Hilton Seattle Airport 380 Upl...,0.873906


In [57]:
matches_df[matches_df['similarity'] < 0.50].right_side.nunique()

150